In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime

%matplotlib inline

In [ ]:
lrpd = pd.read_csv("../data/lrpd-clean.csv")

In [ ]:
lrpd.head().T

In [ ]:
lrpd.tail().T

### Question: What Year had the most amount of crime, and how many crimes occurred that year?

In [ ]:
yearly_crimes = pd.DataFrame({'Year': [], 'Crimes': []})

In [ ]:
lrpd["INCIDENT_DATE"] = pd.to_datetime(lrpd["INCIDENT_DATE"])

In [ ]:
for year in lrpd["INCIDENT_DATE"].dt.year.unique():
    yearly_data = lrpd.loc[lrpd["INCIDENT_DATE"].dt.year == year]
    yearly_crime = yearly_data["OFFENSE_DESCRIPTION"].value_counts().sum()

    yearly_crimes.loc[len(yearly_crimes.index)] = [year, yearly_crime]

In [ ]:
yearly_crimes = yearly_crimes.set_index("Year").sort_index()
yearly_crimes

In [ ]:
sns.barplot(data=yearly_crimes, x=yearly_crimes.index, y=yearly_crimes["Crimes"])
plt.title("Amount of Crimes-per-Year in Little Rock, AR.")
plt.xlabel("Year")
plt.ylabel("Crimes")
plt.show()
plt.show()

### Answer: 2017, ~16,000

### Question: What were the most common Violent Crimes? Non-Violent?

In [ ]:
violent_crime_occurrence = pd.DataFrame({ "Crime": [], "Occurrence": [] })
nonviolent_crime_occurrence = pd.DataFrame({ "Crime": [], "Occurrence": [] })

violent_offenses = lrpd.loc[lrpd["CRIME_TYPE"] == "Violent Crime"]
for offense in violent_offenses["OFFENSE_DESCRIPTION"].unique():
    occurrence = violent_offenses.loc[violent_offenses["OFFENSE_DESCRIPTION"] == offense].value_counts().sum()

    violent_crime_occurrence.loc[len(violent_crime_occurrence.index)] = [offense, occurrence]

nonviolent_offenses = lrpd.loc[lrpd["CRIME_TYPE"] == "Non-Violent Crime"]
for offense in nonviolent_offenses["OFFENSE_DESCRIPTION"].unique():
    occurrence = nonviolent_offenses.loc[nonviolent_offenses["OFFENSE_DESCRIPTION"] == offense].value_counts().sum()

    nonviolent_crime_occurrence.loc[len(nonviolent_crime_occurrence.index)] = [offense, occurrence]

In [ ]:
ax = sns.barplot(data=violent_crime_occurrence, x=["Rape", "Larceny", "Robbery", "Burglary", "Agg. Assault", "Murder & Manslaughter"], y=violent_crime_occurrence["Occurrence"])
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
plt.title("Offense to Offense Occurrence for Violent Crimes in Little Rock, AR.")
plt.xlabel("Offense")
plt.ylabel("Offense Occurrence")
plt.show()

In [ ]:
ax = sns.barplot(data=nonviolent_crime_occurrence, x=["Theft from MV", "Theft of MV Parts", "Shoplifting", "Theft from Building", "MV Theft", "Purse-Snatching", "Pocket-Picking", "Theft from Coin-Op. Machine"], y=nonviolent_crime_occurrence["Occurrence"])
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
plt.title("Offense to Offense Occurrence for Non-Violent Crimes in Little Rock, AR.")
plt.xlabel("Offense")
plt.ylabel("Offense Occurrence")
plt.show()

### Answer: Violent - Larceny, Non-Violent - Theft from MV

### Question: What Zip had the most crimes?

In [ ]:
zips = lrpd["ZIP"].unique()
zips

In [ ]:
zip_offense_counts = pd.DataFrame({ "Zip": [], "Offense Count": []})

for zip in zips:
    data = lrpd.loc[lrpd["ZIP"] == zip]
    crimes = data["OFFENSE_DESCRIPTION"].value_counts().sum()

    zip_offense_counts.loc[len(zip_offense_counts.index)] = [zip, crimes]

zip_offense_counts.set_index("Zip").sort_values(by="Offense Count")

In [ ]:
ax = sns.barplot(data=zip_offense_counts, x=zip_offense_counts["Zip"], y=zip_offense_counts["Offense Count"])
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

plt.title("Offense Counts of Zipcodes in Little Rock, AR.")
plt.xlabel("Zip")
plt.ylabel("Offense Count")
plt.show()

### Answer: Zip 72209 had 16k+ crimes.

### Question: What weapons were used the most in Violent Crimes?

### Seasonality of crimes (According to day of the year), from 2017 to 2021

In [ ]:
years = lrpd["YEAR"].unique()

for year in years:
    crimesInYear = lrpd.loc[lrpd["YEAR"] == year]
    crimesPerDayInYear = crimesInYear.groupby(["MONTH", "DAY"]).count()["INCIDENT_NUMBER"]

    fig, ax = plt.subplots(figsize=(10, 10))
    lineplt = crimesPerDayInYear.plot(kind='line', ax=ax, xlabel="Month, Day Range", ylabel="Amount of Crimes per Day")
    plt.title(year)

### Interactive Map of crimes from 2017 to 2022

In [ ]:
#Create heatmap of crimes in Little Rock
from collections import defaultdict, OrderedDict

data = defaultdict(list)

lrpd_lat_long = lrpd.dropna(subset=["LATITUDE", "LONGITUDE"])

for r in lrpd_lat_long.itertuples():
    data[r.YEAR].append([r.LATITUDE, r.LONGITUDE])

data = OrderedDict(sorted(data.items(), key=lambda t: t[0]))

# Available map tile types in Folium:
    # “OpenStreetMap”
    # “Mapbox Bright” (Limited levels of zoom for free tiles)
    # “Mapbox Control Room” (Limited levels of zoom for free tiles)
    # “Stamen” (Terrain, Toner, and Watercolor)
    # “Cloudmade” (Must pass API key)
    # “Mapbox” (Must pass API key)
    # “CartoDB” (positron and dark_matter)

m = folium.Map(location=[34.7466, -92.2896], zoom_start=12, tiles="OpenStreetMap")
m.add_child(folium.LatLngPopup())

hm = HeatMapWithTime(data=list(data.values()),
                     index=list(data.keys()),
                     radius=10,
                     auto_play=True,
                     max_opacity=0.3)
hm.add_to(m)
m.save("../visualizations/folium-map.html")

m